# Monte Carlo uncertainty simulation

Steps to propagate biomass uncertainty through Monte Carlo simulation. Based on the proposal by Réjou-Méchain et al., 2017, Methods in Ecology and Evolution 8:1163-1167 and implemented in the R package Biomass.

Steps to propagate biomass uncertainty through Monte Carlo simulation.

1. Model tree diameter error. Could be normal distribution, parameters fitted from quality control measurements.  

2. Model wood density error. Biomass uses a truncated normal distribution based on the absolute ranges recorded at the wood density database (0.08-1.39 g/ml). A better way could be based on ancestral range reconstructed on a seed plant phylogeny.

3. Tree height error. Biomass takes a truncated normal distribution with the range(1.3-(maximum_height + 15)). However, this kind of error seems to by exponentially distributed, as it is more likely to record wrong values with taller trees. Parameters could be fitted with quality control measurements.

4. Allometric equation uncertainty. Depending on the allometric equation employed, coefficient distributions are estimated. Biomass estimates a posterior distribution for each equation using a MCMCMC.

5. Above ground biomass estimates are simulated for each tree n times using all the parameter distributions presented above, plus a random error. 


In [ ]:
import pandas as pd
import numpy as np
import pymc3
%matplotlib

In [ ]:
# Sample tree diameter and wood density uncertainty values
diam = 35.0
sdd = diam / 100.0
diams = np.random.normal(diam, sdd, 100)

wd = 0.5
sdwd = 0.01
wds = np.random.normal(wd, sdwd, 100)

zip(diams,wds)

In [ ]:
# Load harvest data from Chave et al 2014
har = pd.read_csv("../Chave_harvest_db/Chave_GCB_Direct_Harvest_Data.csv")
loc = pd.read_csv("../Chave_harvest_db/Localities.csv")
loc['Forest_type'] = loc.Forest_type.str.replace(' forest','')
#print har.columns
#print loc.columns

In [ ]:
def est_E(row):
    #E = ( 0.178 × TS-0.938 × CWD-6.61× PS ) ×10 −3
    ts = loc[loc.Abbreviation == row.Site]['Temp_Seasonality'].item()
    cwd = loc[loc.Abbreviation == row.Site]['CWD_mm_yr'].item()
    ps = loc[loc.Abbreviation == row.Site]['Precip_Seasonality_perc'].item()
    E = (0.178 * ts - 0.938 * cwd - 6.61 * ps) * 1e-3
    return E

har['E'] = har.apply(est_E, axis=1)

In [ ]:
mymodel = pymc3.Model()
trace = None
with mymodel:
    #Priors
    a = pymc3.Normal('a', mu = -2.109, sd = 0.5)
    b = pymc3.Normal('b', mu = -0.896, sd = 0.5)
    c = pymc3.Normal('c', mu = 0.923, sd = 0.5)
    d = pymc3.Normal('d', mu = 2.794, sd = 0.5)
    e = pymc3.Normal('e', mu = -0.046, sd = 0.5)
    sigma = pymc3.HalfNormal('sigma', sd=1)
    
    # Allometric equation Chave 2014
    y_exp = a + b * har.E + c * np.log(har.Gravity) + d * np.log(har.DBH_cm) + e * np.log(har.DBH_cm)**2   
        
    # Likelihood function
    Y_obs = pymc3.Normal('Y_obs', mu=y_exp, sd=sigma, observed=np.log(har.AGB_kg))
    
    # Metropolis kernel
    #mstep = pymc3.Metropolis()
    #trace = pymc3.sample(50000, njobs=4, step=mstep)
    
    # NUTS kernel is default option for continuous parameters
    trace = pymc3.sample(5000, njobs=4)


In [ ]:
pymc3.diagnostics.effective_n(trace)

In [ ]:
pymc3.summary(trace)

In [ ]:
pymc3.traceplot(trace)